In [ ]:
import pandas as pd
import numpy as np
from functools import reduce

## Making grand_dict 
w/ keys: filename, delimeter, encoding, filetype, grouping

In [ ]:
location = '../Market_Data-Municipality_Level-20210915/'
file_info = '''
life_expectancy.csv ; latin_1 brick
T2D.csv ; latin_1 brick
Sales.csv , utf-8 other
longterm_disease_.csv ; latin_1 brick
andel_blodprop_i_hjertet.csv ; latin_1 brick
patient_data.csv , utf-8 other
attention.csv , utf-8 other
svaert_overvaegt.csv ; latin_1 brick
attention_3.csv ; utf-8 other
svært_overvægt.csv ; latin_1 #
attention_3.xls ; utf-8 #
uddannelse_lang.csv ; latin_1 brick
fravaer.csv ; latin_1 brick
variables_explanations.xlsx ; utf-8 #
fravær.csv ; latin_1 #
who_minimums_fysisk_aktivitet.csv ; latin_1 brick'''.split('\n')[1:]
file_info = [location+i for i in file_info]

In [ ]:
grand_dict = {}
for i in file_info:
    contents = i.split(' ')
    #print(contents)
    if contents[3] == '#':
        pass
    else:
        name = contents[0].split('/')[2].split('.')[0]
        grand_dict[name] = {
            'filename' :contents[0],
            'delimeter': contents[1],
            'encoding':contents[2],
            'filetype': contents[0].split('/')[2].split('.')[1],
            'grouping' :contents[3]}
grand_dict['life_expectancy']

In [ ]:
for i in grand_dict.keys():
    if grand_dict[i]['filetype'] == 'csv':
        grand_dict[i]['dataframe'] = pd.read_csv(grand_dict[i]['filename'], 
                                                 delimiter=grand_dict[i]['delimeter'], 
                                                 encoding=grand_dict[i]['encoding'])

## Dropping and renaming cols

In [ ]:
grand_dict['life_expectancy']['dataframe'].rename(columns={'år':'life_expectancy (yrs)'}, inplace=True)

grand_dict['T2D']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['longterm_disease_']['dataframe'].rename(columns={'anddel':'longterm_disease_anddel'}, inplace=True)
grand_dict['longterm_disease_']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['andel_blodprop_i_hjertet']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['patient_data']['dataframe'].drop(columns=['Unnamed: {}'.format(i) for i in range(4,13)],inplace=True)

grand_dict['svaert_overvaegt']['dataframe'].drop(columns=['brick'],inplace=True)
grand_dict['svaert_overvaegt']['dataframe'].rename(columns={'Andel':'svaert_overvaegt Andel'}, inplace=True)

grand_dict['attention_3']['dataframe'].drop(columns=['BRICK_NAME'],inplace=True)
grand_dict['attention_3']['dataframe'].dropna(inplace=True)
grand_dict['attention_3']['dataframe'].rename(columns={'BRICK_NO':'brick_nr'}, inplace=True)
grand_dict['attention_3']['dataframe'] = grand_dict['attention_3']['dataframe'][grand_dict['attention_3']['dataframe']['brick_nr'] != 'Outside DK']
grand_dict['attention_3']['dataframe']['brick_nr'] = grand_dict['attention_3']['dataframe']['brick_nr'].apply(lambda x: int(x))

grand_dict['uddannelse_lang']['dataframe'].drop(columns=['brick', 'brick_navn', 'population'],inplace=True)
grand_dict['uddannelse_lang']['dataframe'].rename(columns={'antal':'uddannelse_lang antal', 'value_per_1000' : 'uddannelse_lang (value_per_1000)'}, inplace=True)

grand_dict['fravaer']['dataframe'].drop(columns=['brick'],inplace=True)

grand_dict['who_minimums_fysisk_aktivitet']['dataframe'].drop(columns=['brick'],inplace=True)
grand_dict['who_minimums_fysisk_aktivitet']['dataframe'].rename(columns={'Andel':'who_minimums_fysisk_aktivitet Andel'}, inplace=True)

## Combining dfs of equal length

In [ ]:
keys = []
for key in grand_dict.keys():
    if grand_dict[key]['grouping'] == 'brick':
        
        keys.append(key)
dfs_brick = [grand_dict[key]['dataframe'] for key in keys]        
df_final = reduce(lambda left,right: pd.merge(left,right,on='brick_nr'), dfs_brick)

## What is missing?
Lets print remaining .csv files and the number of unique elements in each column.

In [ ]:
for key in grand_dict.keys():
    if grand_dict[key]['grouping'] != 'brick':
        print(key)
        for col in grand_dict[key]['dataframe'].columns:
            print(col, grand_dict[key]['dataframe'][col].nunique())
        print(len(grand_dict[key]['dataframe'][col]),'\n')

### attention_3

In [ ]:
# we should be able to get attention_3 to fit in the brick group
new_attention_3 = df_final[['brick_nr']].copy()
new_attention_3['fake'] = np.zeros(len(new_attention_3))
df = grand_dict['attention_3']['dataframe'].copy()
for WHO_ATC_CODE in df['WHO_ATC_CODE'].unique():
    for PERIOD_ID in df['PERIOD_ID'].unique():
        for SITE_SHORT in df['SITE_SHORT'].unique():
            temp = df[(df['WHO_ATC_CODE'] == WHO_ATC_CODE) & (df['PERIOD_ID'] == PERIOD_ID) & (df['SITE_SHORT'] == SITE_SHORT)].copy()
            name = WHO_ATC_CODE+'_'+str(PERIOD_ID)+'_'+SITE_SHORT + '_COUNT'
            temp.drop(columns=['WHO_ATC_CODE', 'PERIOD_ID', 'SITE_SHORT', 'CATEGORY'], inplace=True)
            temp.rename(columns={'COUNT_':name}, inplace=True)
            new_attention_3 = new_attention_3.join(temp.set_index('brick_nr'), on='brick_nr')

new_attention_3.drop(columns=['fake'], inplace=True)
#new_attention_3.head()

In [ ]:
dic_min = {}
dic_pro = {}

df = grand_dict['attention_3']['dataframe'].copy()
for i in df.index[:]:
    WHO_ATC_CODE = df['WHO_ATC_CODE'][i]
    brick_nr = df['brick_nr'][i]
    SITE_SHORT = df['SITE_SHORT'][i]
    PERIOD_ID = df['PERIOD_ID'][i]
    
    COUNT_ = df['COUNT_'][i]
    #print('\n',df.iloc[i])
    
    if SITE_SHORT == 'min':
        dic = dic_min
    else:
        dic = dic_pro
    
    if WHO_ATC_CODE in dic.keys():
        if brick_nr in dic[WHO_ATC_CODE].keys():
            dic[WHO_ATC_CODE][brick_nr][PERIOD_ID] = COUNT_
        else:
            dic[WHO_ATC_CODE][brick_nr] = {PERIOD_ID: COUNT_}
    else:
        dic[WHO_ATC_CODE] = {brick_nr : {PERIOD_ID: COUNT_}}
        
    if SITE_SHORT == 'min':
        dic_min = dic
    else:
        dic_pro = dic
        
        
dic_min['A10AE04'][502]

In [ ]:
dict_of_df = {k: pd.DataFrame(v) for k,v in dic_min.items()}
df_attention_3_min = pd.concat(dict_of_df, axis=1).T

dict_of_df = {k: pd.DataFrame(v) for k,v in dic_pro.items()}
df_attention_3_pro = pd.concat(dict_of_df, axis=1).T

#df_attention_3_min # /min

### Sales

In [ ]:
# Now for sales

# First lets grab a copy
df = grand_dict['Sales']['dataframe'].copy()
df = df[df['SECTOR_CODE']=='P']
df.drop(columns=['SECTOR_CODE'], inplace=True)
value_dict = {}
volume_dict = {}

for i in df.index:
    MUNICIPALITY, WHO_ATC_CODE, YEAR_MONTH = df['MUNICIPALITY'][i], df['WHO_ATC_CODE'][i], df['YEAR_MONTH'][i]
    VOLUME, VALUE = df['VOLUME'][i], df['VALUE'][i]
    
    if WHO_ATC_CODE in value_dict.keys():
        if MUNICIPALITY in value_dict[WHO_ATC_CODE].keys():
            value_dict[WHO_ATC_CODE][MUNICIPALITY][YEAR_MONTH] =VALUE
            volume_dict[WHO_ATC_CODE][MUNICIPALITY][YEAR_MONTH] =VOLUME   
        else:
            value_dict[WHO_ATC_CODE][MUNICIPALITY] = {YEAR_MONTH :VALUE}
            volume_dict[WHO_ATC_CODE][MUNICIPALITY] = {YEAR_MONTH :VOLUME}
    else:
        for dic, x in zip([value_dict, volume_dict],[VALUE, VOLUME]):
            dic[WHO_ATC_CODE] = {}
            dic[WHO_ATC_CODE][MUNICIPALITY]= {YEAR_MONTH: x}
            
            
dict_of_df = {k: pd.DataFrame(v) for k,v in value_dict.items()}
df_value = pd.concat(dict_of_df, axis=1)

dict_of_df = {k: pd.DataFrame(v) for k,v in volume_dict.items()}
df_volume = pd.concat(dict_of_df, axis=1)

#df_value.T.head(3)
#df_volume.T#.head(3)

### patient_data

In [ ]:
df = grand_dict['patient_data']['dataframe'].copy()
df_patient_data = pd.DataFrame(df[['n_type1','n_type2']].values, 
                   index=[
                       df['Hospital'].values,
                       df['Department'].values], 
                   columns='n_type1, n_type2'.split(', '))
#df_patient_data.head()

### attention

In [ ]:
df = grand_dict['attention']['dataframe'].copy()
df['Region'] = df['REGIONNAME']+' '+ df['REGION'].astype(str)
df['City'] = df['ZIP'].astype(str)+' '+ df['CITY']
df.drop(columns='CITY, REGION, REGIONNAME, ZIP'.split(', '), inplace=True)

#df.head()

In [ ]:
clicks_dict_pro = {} # ZIP = {}, 
clicks_dict_min = {}

for i in df.index[:]:
    Region,City , WHO_ATC_CODE, YEAR_MONTH = df['Region'][i], df['City'][i], df['WHO_ATC_5_DERIVED'][i], df['PERIOD_YEAR_MONTH_NUM'][i]
    clicks, SITE_SHORT = df['CLICKS'][i], df['SITE_SHORT'][i]
    #print(Region)
    if SITE_SHORT == 'pro':
        if Region in clicks_dict_pro.keys():
            if City in clicks_dict_pro[Region].keys():
                if WHO_ATC_CODE in clicks_dict_pro[Region][City].keys():
                    clicks_dict_pro[Region][City][WHO_ATC_CODE][YEAR_MONTH] = clicks
                else:
                    clicks_dict_pro[Region][City][WHO_ATC_CODE] = {YEAR_MONTH: clicks}
            else:
                clicks_dict_pro[Region][City] = {WHO_ATC_CODE :{YEAR_MONTH :clicks}}
        else:
            clicks_dict_pro[Region] = {}
            clicks_dict_pro[Region][City]= {WHO_ATC_CODE: {YEAR_MONTH : clicks}}
    else:
        if Region in clicks_dict_min.keys():
            if City in clicks_dict_min[Region].keys():
                if WHO_ATC_CODE in clicks_dict_min[Region][City].keys():
                    clicks_dict_min[Region][City][WHO_ATC_CODE][YEAR_MONTH] = clicks
                else:
                    clicks_dict_min[Region][City][WHO_ATC_CODE] = {YEAR_MONTH: clicks}
            else:
                clicks_dict_min[Region][City] = {WHO_ATC_CODE :{YEAR_MONTH :clicks}}
        else:
            clicks_dict_min[Region] = {}
            clicks_dict_min[Region][City]= {WHO_ATC_CODE: {YEAR_MONTH : clicks}}
#clicks_dict_pro
            


In [ ]:
reform = {(level1_key, level2_key, level3_key): values
   ...:           for level1_key, level2_dict in clicks_dict_min.items()
   ...:           for level2_key, level3_dict in level2_dict.items()
   ...:           for level3_key, values      in level3_dict.items()}
df_attention_clicks_min = pd.DataFrame(reform).T

reform = {(level1_key, level2_key, level3_key): values
   ...:           for level1_key, level2_dict in clicks_dict_pro.items()
   ...:           for level2_key, level3_dict in level2_dict.items()
   ...:           for level3_key, values      in level3_dict.items()}
df_attention_clicks_pro = pd.DataFrame(reform).T

## Now lets save all the files

In [ ]:
df_value.T.to_csv('../Cleaned_Market_data_anton/df_value.csv')
df_volume.T.to_csv('../Cleaned_Market_data_anton/df_volume.csv')

In [ ]:
loc = '../antonsdata/'

# bricks
df_final.to_csv(loc+'df_bricks.csv')

# Attention 3
df_attention_3_min.to_csv(loc+'new_attention_3_min.csv')
df_attention_3_pro.to_csv(loc+'new_attention_3_pro.csv')

# Sales
df_value.T.to_csv(loc+'df_value.csv')
df_volume.T.to_csv(loc+'df_volume.csv')

# Patient data
df_patient_data.to_csv(loc+'df_patient_data.csv')

# attention
df_attention_clicks_min.to_csv(loc+'df_attention_clicks_min.csv')
df_attention_clicks_pro.to_csv(loc+'df_attention_clicks_pro.csv')

## And try loading them again

In [ ]:
# bricks
dfA = pd.read_csv(loc+'df_bricks.csv',index_col=[0],decimal= ',')

# Attention 3
dfB1 = pd.read_csv(loc+'new_attention_3_pro.csv',index_col=[0,1])
dfB2 = pd.read_csv(loc+'new_attention_3_min.csv',index_col=[0,1])

# Sales
dfC1 = pd.read_csv(loc+'df_value.csv',index_col=[0,1])
dfC2 = pd.read_csv(loc+'df_volume.csv',index_col=[0,1])

# Patient data
dfD = pd.read_csv(loc+'df_patient_data.csv',index_col=[0,1])

# attention

dfE1 = pd.read_csv(loc+'df_attention_clicks_min.csv',index_col=[0,1,2])
dfE2 = pd.read_csv(loc+'df_attention_clicks_pro.csv',index_col=[0,1,2])